In [1]:
import os
import random
import shutil
import SimpleITK as sitk
import numpy as np

# ---------------------------
# 1. Split the Data into 5 Folds
# ---------------------------
# Set a seed for reproducibility
random.seed(42)

# Define the path to the directory containing the patient folders.
train_dir = './dataset/train'  # Adjust this path if necessary

# List all folders that start with "patient" and are directories.
patient_dirs = [d for d in os.listdir(train_dir) 
                if d.startswith("patient") and os.path.isdir(os.path.join(train_dir, d))]
patient_dirs.sort()  # Ensure a consistent order before shuffling

print("Total number of patient folders:", len(patient_dirs))

# Shuffle the patient folders to randomize the split
random.shuffle(patient_dirs)

# Compute the number of folds
num_folds = 5
fold_size = len(patient_dirs) // num_folds

# Create directories for each fold
for fold in range(num_folds):
    fold_dir = f'./dataset/fold_{fold}'
    os.makedirs(fold_dir, exist_ok=True)

    # Select patient folders for this fold
    start_index = fold * fold_size
    end_index = start_index + fold_size
    fold_patients = patient_dirs[start_index:end_index]

    print(f"Fold {fold}:")
    print("Number of patients in this fold:", len(fold_patients))

    # Copy patient folders to the fold directory
    for patient in fold_patients:
        src = os.path.join(train_dir, patient)
        dst = os.path.join(fold_dir, patient)
        shutil.copytree(src, dst, dirs_exist_ok=True)

    print(f"Data for fold {fold} copied to: {fold_dir}")

print("\nData splitting into folds complete.")

# ---------------------------
# 2. Resample the Images to 1.25 x 1.25 mm In-Plane Resolution
# ---------------------------
# Set the target in-plane resolution (width and height) in mm.
target_resolution = [1.25, 1.25]

def resample_image(image, target_resolution, interpolator=sitk.sitkLinear):
    """
    Resample a 2D or 3D image so that its first two dimensions (x and y) have the specified resolution.
    The third dimension (if present) is left unchanged.
    """
    original_spacing = image.GetSpacing()   # e.g., (sx, sy, [sz])
    original_size = image.GetSize()           # e.g., (nx, ny, [nz])
    
    # Compute the new size for x and y
    new_size = list(original_size)
    new_size[0] = int(round(original_size[0] * original_spacing[0] / target_resolution[0]))
    new_size[1] = int(round(original_size[1] * original_spacing[1] / target_resolution[1]))
    
    # Create new spacing: update x and y; leave others unchanged.
    new_spacing = list(original_spacing)
    new_spacing[0] = target_resolution[0]
    new_spacing[1] = target_resolution[1]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetInterpolator(interpolator)
    
    return resampler.Execute(image)

# Process each fold for resampling
for fold in range(num_folds):
    fold_dir = f'./dataset/fold_{fold}'

    if not os.path.exists(fold_dir):
        print("Directory does not exist:", fold_dir)
        continue
    
    print("Processing directory:", fold_dir)
    for patient in os.listdir(fold_dir):
        patient_path = os.path.join(fold_dir, patient)
        if os.path.isdir(patient_path):
            for file in os.listdir(patient_path):
                if 'frame' in file and '4d' not in file and file.endswith('.nii'):
                    file_path = os.path.join(patient_path, file)
                    image = sitk.ReadImage(file_path)
                    spacing = image.GetSpacing()
                    if abs(spacing[0] - target_resolution[0]) < 1e-6 and abs(spacing[1] - target_resolution[1]) < 1e-6:
                        print("Skipping already resampled:", file_path)
                        continue
                    
                    print("Resampling:", file_path)
                    interp = sitk.sitkNearestNeighbor if '_gt' in file else sitk.sitkLinear
                    resampled_image = resample_image(image, target_resolution, interpolator=interp)

                    # Attempt to write the image
                    try:
                        sitk.WriteImage(resampled_image, file_path)
                    except Exception as e:
                        print(f"Error writing image {file_path}: {e}")

print("Resampling for all folds complete.")

# ---------------------------
# 3. Intensity Normalization (Z-score)
# ---------------------------
def zscore_normalization(image):
    """
    Normalize the image using Z-score normalization:
      (image - mean) / std
    """
    # Convert the image to a NumPy array.
    arr = sitk.GetArrayFromImage(image)
    mean_val = np.mean(arr)
    std_val = np.std(arr)
    
    # Prevent division by zero.
    if std_val == 0:
        std_val = 1
    
    normalized_arr = (arr - mean_val) / std_val
    
    # Convert the normalized array back to a SimpleITK image and preserve metadata.
    normalized_image = sitk.GetImageFromArray(normalized_arr)
    normalized_image.CopyInformation(image)
    return normalized_image

# Process each fold for normalization
for fold in range(num_folds):
    fold_dir = f'./dataset/fold_{fold}'

    if not os.path.exists(fold_dir):
        print("Directory does not exist:", fold_dir)
        continue

    print("Processing directory for normalization:", fold_dir)
    for patient in os.listdir(fold_dir):
        patient_path = os.path.join(fold_dir, patient)
        if os.path.isdir(patient_path):
            for file in os.listdir(patient_path):
                if 'frame' in file and '4d' not in file and file.endswith('.nii'):
                    if '_gt' not in file:
                        file_path = os.path.join(patient_path, file)
                        print("Normalizing:", file_path)
                        image = sitk.ReadImage(file_path)
                        normalized_image = zscore_normalization(image)
                        sitk.WriteImage(normalized_image, file_path)

print("Intensity normalization complete for all folds.")

# ---------------------------
# 4. Resize Images to 352 x 352 Pixels for 2D U-Net Input
# ---------------------------
# Desired output size for the network input.
target_size = (352, 352)

def resize_image_to_fixed_size(image, output_size, interpolator=sitk.sitkLinear):
    """
    Resize a 2D (or 3D with a single slice dimension) image to a fixed output size.
    For 3D images, only the first two dimensions (x, y) are resized and the slice dimension is kept unchanged.
    """
    dim = image.GetDimension()
    original_size = list(image.GetSize())
    original_spacing = list(image.GetSpacing())
    
    if dim == 2:
        # Compute new spacing for a 2D image.
        new_size = list(output_size)
        new_spacing = [(original_size[i] * original_spacing[i]) / output_size[i] for i in range(2)]
    elif dim == 3:
        # For a 3D image, assume the third dimension is the slice dimension.
        new_size = list(output_size) + [original_size[2]]
        new_spacing_xy = [(original_size[i] * original_spacing[i]) / output_size[i] for i in range(2)]
        new_spacing = new_spacing_xy + [original_spacing[2]]
    else:
        raise ValueError("Image dimension not supported.")
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(new_size)
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetInterpolator(interpolator)
    
    return resampler.Execute(image)

# Process each fold for resizing
for fold in range(num_folds):
    fold_dir = f'./dataset/fold_{fold}'

    if not os.path.exists(fold_dir):
        print("Directory does not exist:", fold_dir)
        continue
        
    print("Processing directory for resizing:", fold_dir)
    for patient in os.listdir(fold_dir):
        patient_path = os.path.join(fold_dir, patient)
        if os.path.isdir(patient_path):
            for file in os.listdir(patient_path):
                if 'frame' in file and '4d' not in file and file.endswith('.nii'):
                    file_path = os.path.join(patient_path, file)
                    print("Resizing:", file_path)
                    image = sitk.ReadImage(file_path)
                    interp = sitk.sitkNearestNeighbor if '_gt' in file else sitk.sitkLinear
                    resized_image = resize_image_to_fixed_size(image, target_size, interpolator=interp)
                    sitk.WriteImage(resized_image, file_path)

print("Resizing complete for all folds.")

# ---------------------------
# 5. Extract 2D Slices from Volumes for 2D U-Net Training
# ---------------------------
def extract_slices(volume_image):
    """
    Extract 2D slices from a 3D image along the z-dimension using SimpleITK's ExtractImageFilter.
    Returns a list of 2D SimpleITK images.
    """
    size = list(volume_image.GetSize())
    slices = []
    extractor = sitk.ExtractImageFilter()
    
    for z in range(size[2]):
        extraction_index = [0, 0, z]
        extraction_size = [size[0], size[1], 0]  # 0 means "extract one slice" along that dimension.
        
        extractor.SetSize(extraction_size)
        extractor.SetIndex(extraction_index)
        slice_img = extractor.Execute(volume_image)
        slices.append(slice_img)
    return slices

# Process each fold to extract 2D slices
for fold in range(num_folds):
    fold_dir = f'./dataset/fold_{fold}'

    if not os.path.exists(fold_dir):
        print("Directory does not exist:", fold_dir)
        continue

    # Create output directory for extracted 2D slices for this fold
    slices_dir = f'./dataset/fold_{fold}_slices'
    os.makedirs(slices_dir, exist_ok=True)

    print(f"Extracting 2D slices from {fold_dir} ...")
    for patient in os.listdir(fold_dir):
        patient_path = os.path.join(fold_dir, patient)
        if os.path.isdir(patient_path):
            out_patient_dir = os.path.join(slices_dir, patient)
            os.makedirs(out_patient_dir, exist_ok=True)
            for file in os.listdir(patient_path):
                if 'frame' in file and file.endswith('.nii'):
                    file_path = os.path.join(patient_path, file)
                    print("Extracting slices from:", file_path)
                    image = sitk.ReadImage(file_path)
                    slices = extract_slices(image)
                    for i, slice_img in enumerate(slices):
                        base_name = os.path.splitext(file)[0]
                        new_filename = f"{base_name}_slice{i:02d}.nii"
                        new_file_path = os.path.join(out_patient_dir, new_filename)
                        sitk.WriteImage(slice_img, new_file_path)
                        print("Saved slice:", new_file_path)

print("Slice extraction complete for all folds.")

Total number of patient folders: 100
Fold 0:
Number of patients in this fold: 20
Data for fold 0 copied to: ./dataset/fold_0
Fold 1:
Number of patients in this fold: 20
Data for fold 1 copied to: ./dataset/fold_1
Fold 2:
Number of patients in this fold: 20
Data for fold 2 copied to: ./dataset/fold_2
Fold 3:
Number of patients in this fold: 20
Data for fold 3 copied to: ./dataset/fold_3
Fold 4:
Number of patients in this fold: 20
Data for fold 4 copied to: ./dataset/fold_4

Data splitting into folds complete.
Processing directory: ./dataset/fold_0
Resampling: ./dataset/fold_0/patient043/patient043_frame01.nii


NiftiImageIO (0x556eaaab23c0): ./dataset/fold_0/patient043/patient043_frame01.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_0/patient043/patient043_frame01.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_0/patient043/patient043_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_0/patient043/patient043_frame01_gt.nii has unexpected scales in sform



Resampling: ./dataset/fold_0/patient043/patient043_frame01_gt.nii
Resampling: ./dataset/fold_0/patient043/patient043_frame07.nii
Resampling: ./dataset/fold_0/patient043/patient043_frame07_gt.nii


NiftiImageIO (0x556eaaab23c0): ./dataset/fold_0/patient043/patient043_frame07.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_0/patient043/patient043_frame07.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_0/patient043/patient043_frame07_gt.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_0/patient043/patient043_frame07_gt.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_0/patient042/patient042_frame01.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_0/patient042/patient042_frame01.nii has unexpected scales in sform



Resampling: ./dataset/fold_0/patient042/patient042_frame01.nii
Resampling: ./dataset/fold_0/patient042/patient042_frame01_gt.nii
Resampling: ./dataset/fold_0/patient042/patient042_frame16.nii


...

Resampling: ./dataset/fold_4/patient082/patient082_frame07_gt.nii
Resampling for all folds complete.
Processing directory for normalization: ./dataset/fold_0
Normalizing: ./dataset/fold_0/patient043/patient043_frame01.nii
Normalizing: ./dataset/fold_0/patient043/patient043_frame07.nii
Normalizing: ./dataset/fold_0/patient042/patient042_frame01.nii
Normalizing: ./dataset/fold_0/patient042/patient042_frame16.nii
Normalizing: ./dataset/fold_0/patient092/patient092_frame01.nii
Normalizing: ./dataset/fold_0/patient092/patient092_frame06.nii
Normalizing: ./dataset/fold_0/patient010/patient010_frame01.nii
Normalizing: ./dataset/fold_0/patient010/patient010_frame13.nii
Normalizing: ./dataset/fold_0/patient066/patient066_frame01.nii
Normalizing: ./dataset/fold_0/patient066/patient066_frame11.nii
Normalizing: ./dataset/fold_0/patient051/patient051_frame01.nii
Normalizing: ./dataset/fold_0/patient051/patient051_frame11.nii
Normalizing: ./dataset/fold_0/patient002/patient002_frame01.nii
Normalizin

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_4/patient098/patient098_frame01.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_4/patient098/patient098_frame01.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_4/patient098/patient098_frame09.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_4/patient098/patient098_frame09.nii has unexpected scales in sform



Normalizing: ./dataset/fold_4/patient005/patient005_frame13.nii
Normalizing: ./dataset/fold_4/patient055/patient055_frame01.nii
Normalizing: ./dataset/fold_4/patient055/patient055_frame10.nii
Normalizing: ./dataset/fold_4/patient076/patient076_frame01.nii
Normalizing: ./dataset/fold_4/patient076/patient076_frame12.nii
Normalizing: ./dataset/fold_4/patient012/patient012_frame01.nii
Normalizing: ./dataset/fold_4/patient012/patient012_frame13.nii
Normalizing: ./dataset/fold_4/patient070/patient070_frame01.nii
Normalizing: ./dataset/fold_4/patient070/patient070_frame10.nii
Normalizing: ./dataset/fold_4/patient087/patient087_frame01.nii
Normalizing: ./dataset/fold_4/patient087/patient087_frame10.nii
Normalizing: ./dataset/fold_4/patient014/patient014_frame01.nii
Normalizing: ./dataset/fold_4/patient014/patient014_frame13.nii
Normalizing: ./dataset/fold_4/patient018/patient018_frame01.nii
Normalizing: ./dataset/fold_4/patient018/patient018_frame10.nii
Normalizing: ./dataset/fold_4/patient029

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_4/patient098/patient098_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_4/patient098/patient098_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_4/patient098/patient098_frame09_gt.nii has unexpected scales in sform

NiftiImageIO (0x556eaaab23c0): ./dataset/fold_4/patient098/patient098_frame09_gt.nii has unexpected scales in sform



Resizing: ./dataset/fold_4/patient005/patient005_frame01_gt.nii
Resizing: ./dataset/fold_4/patient005/patient005_frame13.nii
Resizing: ./dataset/fold_4/patient005/patient005_frame13_gt.nii
Resizing: ./dataset/fold_4/patient055/patient055_frame01.nii
Resizing: ./dataset/fold_4/patient055/patient055_frame01_gt.nii
Resizing: ./dataset/fold_4/patient055/patient055_frame10.nii
Resizing: ./dataset/fold_4/patient055/patient055_frame10_gt.nii
Resizing: ./dataset/fold_4/patient076/patient076_frame01.nii
Resizing: ./dataset/fold_4/patient076/patient076_frame01_gt.nii
Resizing: ./dataset/fold_4/patient076/patient076_frame12.nii
Resizing: ./dataset/fold_4/patient076/patient076_frame12_gt.nii
Resizing: ./dataset/fold_4/patient012/patient012_frame01.nii
Resizing: ./dataset/fold_4/patient012/patient012_frame01_gt.nii
Resizing: ./dataset/fold_4/patient012/patient012_frame13.nii
Resizing: ./dataset/fold_4/patient012/patient012_frame13_gt.nii
Resizing: ./dataset/fold_4/patient070/patient070_frame01.nii
